In [27]:
import numpy as np
import pandas as pd

def topsis(data, weights, criteria):
    # Step 1: Normalize the Decision Matrix
    norm_matrix = data / np.sqrt((data ** 2).sum(axis=0))
    
    # Step 2: Multiply by Weights
    weighted_matrix = norm_matrix * weights
    
    # Step 3: Determine Ideal & Negative-Ideal Solutions
    ideal_best = np.where(criteria == 'max', weighted_matrix.max(axis=0), weighted_matrix.min(axis=0))
    ideal_worst = np.where(criteria == 'max', weighted_matrix.min(axis=0), weighted_matrix.max(axis=0))
    
    # Step 4: Compute Separation Measures
    distance_best = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))
    
    # Step 5: Compute TOPSIS Score
    scores = distance_worst / (distance_best + distance_worst)
    
    return scores

In [29]:
data = pd.DataFrame({
    "Model": ["GPT-3", "BERT-GPT", "T5", "GPT-Neo"],
    "Perplexity": [12.5, 14.2, 11.8, 13.1],  # Lower is better
    "BLEU": [40.2, 38.7, 42.3, 39.5],  # Higher is better
    "ROUGE": [45.8, 43.2, 47.1, 44.7],  # Higher is better
    "Training Time": [10, 8, 12, 9]  # Lower is better
})
data.head()

,Model,Perplexity,BLEU,ROUGE,Training Time
0,GPT-3,12.5,40.2,45.8,10
1,BERT-GPT,14.2,38.7,43.2,8
2,T5,11.8,42.3,47.1,12
3,GPT-Neo,13.1,39.5,44.7,9


In [31]:
# Convert to numpy array (excluding Model names)
data_values = data.iloc[:, 1:].values

# Define weights (sum should be 1)
weights = np.array([0.3, 0.3, 0.3, 0.1])

# Define criteria: 'max' for benefit, 'min' for cost
criteria = np.array(['min', 'max', 'max', 'min'])

In [33]:
# Compute TOPSIS scores
topsis_scores = topsis(data_values, weights, criteria)

# Add scores to DataFrame
data["TOPSIS Score"] = topsis_scores

# Rank models based on scores
data = data.sort_values(by="TOPSIS Score", ascending=False)

# Print final ranking
print(data)

      Model  Perplexity  BLEU  ROUGE  Training Time  TOPSIS Score
2        T5        11.8  42.3   47.1             12      0.622977
0     GPT-3        12.5  40.2   45.8             10      0.607802
3   GPT-Neo        13.1  39.5   44.7              9      0.500639
1  BERT-GPT        14.2  38.7   43.2              8      0.377023
